In [10]:
import numpy as np
import scipy as sp
import sympy as sympy

### Question 16

In [ ]:
import numpy as np
from scipy.linalg import expm

def build_generator_Q(k: int = 12):

    S = np.array([
        [-1.0,   1.0,  0.0,  0.0],
        [ 0.0,  -1.0,  0.9,  0.1],
        [ 0.0,   0.0, -1.0,  0.0],
        [ 0.0,   0.0,  0.0, -1.0/11.0]
    ])

    s = np.array([0.0, 0.0, 1.0, 1.0/11.0])
    alpha = np.array([1.0, 0.0, 0.0, 0.0])
    Sa = np.outer(s, alpha)

    n_levels = k
    dim = 4 * n_levels + 1
    absorbing = dim - 1

    Q = np.zeros((dim, dim))

    for n in range(n_levels):
        Q[4*n:4*n+4, 4*n:4*n+4] = S
        if n < n_levels - 1:
            Q[4*n:4*n+4, 4*(n+1):4*(n+1)+4] = Sa
        else:
            Q[4*n:4*n+4, absorbing] = s

    return Q
def prob_N_less_than_k(t_months: float = 60.0, k: int = 12) -> float:

    Q = build_generator_Q(k=k)
    dim = Q.shape[0]
    absorbing = dim - 1

    p0 = np.zeros(dim)
    p0[0] = 1.0

    pt = p0 @ expm(Q * t_months)
    return 1.0 - pt[absorbing]


t_months = 60.0  # 5 years
k = 12   
p = prob_N_less_than_k(t_months=t_months, k=k)
print(f"P(N(5 years) < 12) = {p:.6f}")


P(N(5 years) < 12) = 0.189481


### Question 17

In [ ]:
import scipy
S = np.array([
    [-1.0,  1.0,  0.0,  0.0],
    [ 0.0, -1.0,  0.9,  0.1],
    [ 0.0,  0.0, -1.0,  0.0],
    [ 0.0,  0.0,  0.0, -1.0/11.0]
])

alpha = np.array([[1.0, 0.0, 0.0, 0.0]]) 
one = np.ones((4, 1))

inv_minus_S = np.linalg.inv(-S)
pi_unnormalized = alpha @ inv_minus_S
pi = pi_unnormalized / (pi_unnormalized @ one)


t = 6.0  # months
surv_R = (pi @ scipy.linalg.expm(S * t) @ one);    # P(R > t)
prob_within_t = 1.0 - surv_R                    # P(R <= t)

print(f"P(next termination within {t} months) =", prob_within_t)


mu (months) = [[4.]]
P(next termination within 6.0 months) = [[0.78879698]]


### Question 18

In [29]:
#Question 18
Q = np.array([
    [-1,      1,      0,      0,      0,      0,      0,      0],
    [ 0,     -1,    0.9,    0.1,      0,      0,      0,      0],
    [0.65,    0,     -1,     0,    0.35,     0,      0,      0],
    [0.65/11, 0,      0, -1/11, 0.35/11,    0,      0,      0],
    [1/12,    0,      0,     0, -13/12,     1,      0,      0],
    [0,     1/12,     0,     0,      0, -13/12,   0.9,    0.1],
    [0,       0,    1/12,    0,      1,      0, -13/12,     0],
    [0,       0,      0,   1/12,   1/11,     0,      0, -(1/11+1/12)]
])


### Question 19

In [35]:
def stationary_ctmc(Q):
    n = Q.shape[0]
    A = Q.T.copy()
    A[-1, :] = 1.0
    b = np.zeros(n)
    b[-1] = 1.0
    pi = np.linalg.solve(A, b)
    return pi

pi = stationary_ctmc(Q)

# termination states are 2,3,6,7 with 0-index. 
term_states = [2, 3, 6, 7]
busy_term_states = [6, 7]  

# termination rate r_i: only the "case completes" transitions.
def termination_rate(i):
    return Q[i, 0] + Q[i, 4]

r = np.array([termination_rate(i) for i in range(8)]) #rates

num = sum(pi[i] * r[i] for i in busy_term_states)
den = sum(pi[i] * r[i] for i in term_states)

p_audit = 0.35
p_busy_at_term = num / den
p_selected_and_skipped = p_audit * p_busy_at_term

print("P(busy at termination) =", p_busy_at_term)
print("P(selected and skipped) =", p_selected_and_skipped)

P(busy at termination) = 0.5153094815767255
P(selected and skipped) = 0.1803583185518539


### Question 20

In [36]:
S = Q.copy() #subgenerator
S[2,4] = 0.0   #replace such that we model the absorbtions from idle state
S[3,4] = 0.0   

alpha = np.zeros(8)
alpha[4] = 1.0 #start in beginning of a busy state

ones = np.ones(8)

# Moments for PH:
U = np.linalg.inv(-S)                
EX  = alpha @ U @ ones
EX2 = 2.0 * (alpha @ (U @ U) @ ones)
VarX = EX2 - EX**2

print("E[X] (months) =", EX)
print("Var[X] (months^2) =", VarX)

E[X] (months) = 23.579110781348106
Var[X] (months^2) = 329.77184703010073
